In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [9]:
class Model(torch.nn.Module):
    def __init__(self, syllable_count, hvc_neurons, HVC_num_layers, syllable_features):
        super(Model, self).__init__()
        
        # psuedo RNN
        self.HVC = nn.Linear((syllable_count * hvc_neurons), hvc_neurons)
        self.RA = nn.Linear(hvc_neurons, syllable_features)

        LMAN_Size = syllable_features * 2

        # Technically, Area_X has RA * HVC amount of neurons, but for simplicity, we will assume that it is just RA * 1
        self.Area_X =  nn.Linear(hvc_neurons, LMAN_Size)
        self.LMAN =  nn.Linear(hvc_neurons, LMAN_Size)

        self.init_weights()

        self.clock = 0 

        self.layer_states = {
            "HVC": torch.zeros(hvc_neurons),
            "RA": torch.zeros(syllable_features),
            "Area_X": torch.zeros(LMAN_Size),
            "LMAN": torch.zeros(LMAN_Size)
        }
        

    def forward_HVC(self, syllable, hvc_neurons):
        x = self.HVC(torch.cat((syllable, hvc_neurons), 0))
        return F.relu(x)
    
    def forward_RA(self, hvc_neurons):
        x = self.RA(hvc_neurons)
        return F.relu(x)
    
    def forward_Area_X(self, hvc_neurons):
        x = self.Area_X(hvc_neurons)
        return F.relu(x)
    
    def forward_LMAN(self, hvc_neurons):
        x = self.LMAN(hvc_neurons)
        return F.relu(x)
    
    def init_weights(self):
        """
        Purpose:
            Freezes some weights because they are not trainable

        Args:
            None

        Returns:
            None
        """

    def stdp_update():
        pass 

    def HVC_X_update():
        pass

In [7]:
class train_env():
    def __init__(self, layer_names):
        '''
        Purpose:
            - Initializes the training environment
            
        Args:
            - Dictionary of layer names and their sizes
        
        Returns:
            - None
        '''
        self.clock = 0
        self.layer_names = {}

        for name in layer_names:
            self.layer_names[name] = torch.zeros(layer_names[name])

        
